In [8]:
! pip install gradio_client
! pip install groq
! pip install transformers

In [71]:
import itertools
import re
import numpy as np
import pandas as pd
# import openai
import pandas as pd
import itertools
# import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client

from groq import Groq
import config

df = pd.read_parquet('Halved-DF.parquet.gzip')

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

# combinations_of_two -> DONE
-> working correctly (assuming desired output format is: list of tuples, containing all possible combinations of two ingredients)

In [3]:
# Updated 5/21/2024 by TJ

def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    
    UPDATES
    5/22/2024 by TJ:
    - Modified it quite a bit so output format is one list of tuples
    - Removed the powerset stuff
    
    Inputs (1): 
    ingredients_input (from: user input in app) =  single string with ingredients separated by commas and a space
    
    Outputs (1): 
    ingredients_combinations (to: data_query) = a list of tuples, containing all possible combinations of 2 (so if it's 4 ingredients, a list of 6 tuples and nothing more)

    '''

    # split into list of individual ingredients
    ingredients_list = []
    ingredients = re.split('\s|,', ingredients_input) 
    [ingredients_list.append(ingredient) for ingredient in ingredients if ingredient != '']
    
    # create list of all ingredient combinations
    ingredients_combinations = []
    combinations = itertools.combinations(ingredients_list, 2)
    ingredients_combinations = list(combinations)

    return ingredients_combinations

### Testing combinations_of_two
4 ingredients -> 6 tuples
<br>5 ingredients -> 10 tuples
<br>6 ingredients -> 15 tuples

In [67]:
# 4 ingredients -> 1 list of 6 unique tuples
ingredients_input = 'chicken, garlic, salt, tomato'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

6
[('chicken', 'garlic'), ('chicken', 'salt'), ('chicken', 'tomato'), ('garlic', 'salt'), ('garlic', 'tomato'), ('salt', 'tomato')]


In [68]:
# 5 ingredients -> 1 list of 10 unique tuples
ingredients_input = 'chicken, garlic, salt, tomato, cheese'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

10
[('chicken', 'garlic'), ('chicken', 'salt'), ('chicken', 'tomato'), ('chicken', 'cheese'), ('garlic', 'salt'), ('garlic', 'tomato'), ('garlic', 'cheese'), ('salt', 'tomato'), ('salt', 'cheese'), ('tomato', 'cheese')]


In [69]:
# 6 ingredients -> 1 list of 15 unique tuples
ingredients_input = 'tomato, beef, onion, salt, garlic, pepper'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

15
[('tomato', 'beef'), ('tomato', 'onion'), ('tomato', 'salt'), ('tomato', 'garlic'), ('tomato', 'pepper'), ('beef', 'onion'), ('beef', 'salt'), ('beef', 'garlic'), ('beef', 'pepper'), ('onion', 'salt'), ('onion', 'garlic'), ('onion', 'pepper'), ('salt', 'garlic'), ('salt', 'pepper'), ('garlic', 'pepper')]


In [25]:
# missing spaces and commas
ingredients_input = 'tomato,beef,  onion chicken, apple'

ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

10
[('tomato', 'beef'), ('tomato', 'onion'), ('tomato', 'chicken'), ('tomato', 'apple'), ('beef', 'onion'), ('beef', 'chicken'), ('beef', 'apple'), ('onion', 'chicken'), ('onion', 'apple'), ('chicken', 'apple')]


In [26]:
# ingredients with higher compatibility

ingredients_input = 'chicken, onion, garlic, cheese'
ingredients_combinations = combinations_of_two(ingredients_input)
print(len(ingredients_combinations))
print(ingredients_combinations)

6
[('chicken', 'onion'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')]


# data_query
Issues:
- output is not including all the combinations for some inputs
- higher compatibility lists seem to return more rows? not sure

In [7]:
def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    '''
    Scores all combinations of two ingredients.
    
    Inputs (1): 
    ingredients_combinations (from: combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
    Outputs (1): 
    df_comb (to: muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    '''
    
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    df_comb = pd.DataFrame(data)
    return df_comb

In [65]:
# # trying to understand / re-create
def data_query(ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    '''
    Scores all combinations of two ingredients.
    
    Inputs (1): 
    ingredients_combinations (from: combinations_of_two) = a list of tuples, containing all possible combinations of 2 as  
    
    Outputs (1): 
    df_comb (to: muse_comb) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    '''
    
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            print(score)
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    print(data)

    df_comb = pd.DataFrame(data)
    return df_comb

### Testing data_query

In [12]:
# 6 tuples -> should return 6 rows

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(chicken, garlic)",[1.1800811339314616]
1,"(chicken, salt)",[1.4353972677154387]
2,"(garlic, salt)",[1.5533159218950603]


In [28]:
# 15 tuples -> should return 15 rows

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(beef, onion)",[1.3407460751179292]
1,"(beef, salt)",[1.3667368322711522]
2,"(beef, garlic)",[1.1385957417039734]
3,"(beef, pepper)",[1.1819136702926785]
4,"(onion, salt)",[2.0670735182650013]
5,"(onion, garlic)",[1.4235409132992736]
6,"(onion, pepper)",[1.4949048817870405]
7,"(salt, garlic)",[1.5533159218950603]
8,"(salt, pepper)",[3.3564504877818235]
9,"(garlic, pepper)",[1.1902734549399014]


In [29]:
# 6 tuples -> should return 6 rows
# high compatiblity

ingredients_combinations = [('chicken', 'onion'), ('chicken', 'garlic'), ('chicken', 'cheese'), ('onion', 'garlic'), ('onion', 'cheese'), ('garlic', 'cheese')]

df_comb = data_query(ingredients_combinations)
df_comb

,Combination,Score
0,"(chicken, onion)",[1.3248796627107513]
1,"(chicken, garlic)",[1.1800811339314616]
2,"(chicken, cheese)",[1.0177501859513405]
3,"(onion, garlic)",[1.4235409132992736]
4,"(onion, cheese)",[1.4120038875676597]
5,"(garlic, cheese)",[1.0944496012124034]


# muse_comb
- output is in desired format (a list of strings) so code will run! However...

<br>Issues:
- the strings are just single letters and not ingredients
- the output format of data_query needs to be confirmed first for proper testing. is it ok that combinations are in a tuple and score is in a list? (maybe do muse_comb later)

In [30]:
def muse_comb(df_comb): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     The function calculates the sum of the "Score" values and returns the three combinations with the largest sums

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
                         
    Inputs (1): 
    df_comb (from: data_query) = a datafrome with columns 'Combination' and 'Score', containing ingredient combinations and their scores as a dataframe  (for 4 ingredients: all 6 combinations and their respective scores)
    
    Output (1):
    ingredients_list (to: recipe_generator) = a list of 3 lists, containing the 3 ingredients combinations with highest scores
    [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
    ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
    ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]
    
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(df_comb)):
        df_comb["Sum"] = df_comb["Score"].apply(calculate_sum)

    max_values = df_comb.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists

### Testing muse_comb
(might need to fix data_query output first)

In [31]:
# 6 tuples

ingredients_combinations = [('chicken', 'garlic'),
 ('chicken', 'salt'),
 ('chicken', 'tomato'),
 ('garlic', 'salt'),
 ('garlic', 'tomato'),
 ('salt', 'tomato')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[['g', 'a', 's'], ['h', 's', 'a', 'c'], ['h', 'g', 'a', 'c']]

In [32]:
# 15 tuples

ingredients_combinations = [('tomato', 'beef'),
 ('tomato', 'onion'),
 ('tomato', 'salt'),
 ('tomato', 'garlic'),
 ('tomato', 'pepper'),
 ('beef', 'onion'),
 ('beef', 'salt'),
 ('beef', 'garlic'),
 ('beef', 'pepper'),
 ('onion', 'salt'),
 ('onion', 'garlic'),
 ('onion', 'pepper'),
 ('salt', 'garlic'),
 ('salt', 'pepper'),
 ('garlic', 'pepper')]

df_comb = data_query(ingredients_combinations)

ingredients_lists = muse_comb(df_comb)
ingredients_lists

[['e', 'a', 's', 'p'], ['n', 'o', 'a', 's'], ['g', 'a', 's']]

# recipe_generator -> DONE

Working correctly! Only issue is client-side seems to go down every once in a while (400 error).

In [76]:
def recipe_generator(ingredients_lists):

    '''
    Takes ingredients_list from muse_comb and returns the actual recipes with titles, ingredients, and directions.

    Updates:
    5/22/2024 by TJ:
    - Changed accumulator and output variable name from recipe_list to recipe_dicts

    Inputs (1):
    ingredients_list (from: muse_comb) = a list of 3 lists, containing the 3 ingredients combinations with highest score

    Outputs (1):
    recipe_list (to: get_scores, final_recipe) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes

    '''


    api_key = config.api_key2
    client = Groq(
    api_key=api_key
    )
    recipe_dicts = []

    if len(ingredients_lists) == 1:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {ingredients_lists} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_dicts.append(recipe_dict)

    else:
      for i in range(len(ingredients_lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {ingredients_lists[i]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_dicts.append(recipe_dict)

    return recipe_dicts

### Testing recipe_generator

In [77]:
ingredients_lists = [['tomato', 'chicken', 'garlic'], ['chicken', 'onion', 'salt'], ['onion', 'salt', 'garlic']]

recipe_list = recipe_generator(ingredients_lists)
recipe_list

[{'title': 'Roasted Chicken with Garlic and Tomatoes',
  'ingredients': '* 1 Pound Boneless, Skinless Chicken Breast\n* 2 Big Tomatoes, Cut into 1-inch Pieces\n* 5 Garlic Cloves, Minced',
  'directions': '1. Preheat oven to 425°F (220°C).\n2. In a large bowl, toss chicken, tomatoes, and garlic with a pinch of salt and pepper.\n3. Spread the mixture on a baking sheet lined with parchment paper.\n4. Roast for 25-30 minutes or until chicken is cooked through and cooked through.\n5. Serve hot.'},
 {'title': 'Savory Chicken and Onion',
  'ingredients': '* 1 lb chicken\n* 1 onion\n* pink salt',
  'directions': '1. Preheat pan over medium heat.\n2. Add onion and sauté until tender.\n3. Add chicken and sprinkle with salt.\n4. Cook until chicken is cooked through.\n5. Serve hot.'},
 {'title': 'Garlic-Flavored Onion',
  'ingredients': '* 1 onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'directions': '1. In a large pan, combine the sliced onion and salt. Cook over medium

# get_scores (new function that used to be random code in app.py)
Output format is OK so the code will run, but...

Issues:
- Scores seem to not differ much for both good recipes and nonsense recipes?

In [40]:
def get_scores(recipe_dicts):
    '''
    Generates the score of each recipe.
    
    Inputs (1):  
    recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    
    Outputs (1):
    scores (to: final_recipes) = a list of 3 integers, containing the scores for each recipe.
    '''

    scores = []
    recipe_direction = []

    for recipe in recipe_dicts:
        if 'directions' in recipe:
            recipe_direction.append(recipe['directions'])
        else:
            recipe_direction.append("")

    for direction in recipe_direction:
        scores.append(model.predict_proba([direction])[0][1])

    return scores

### Testing get_scores

In [59]:
# good recipe

recipe_list = [{'title': 'Garlic Chicken Tomato Skillet',
  'ingredients': '* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  'directions': '1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.'},
 {'title': 'Simple Grilled Chicken',
  'ingredients': '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  'directions': '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 minutes per side, or until cooked through.\n4. Meanwhile, grill onion slices until caramelized and slightly charred.\n5. Serve chicken with grilled onion.'},
 {'title': 'Caramelized Onion Relish',
  'ingredients': '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'directions': '1. In a large saucepan, combine the sliced onion, salt, and garlic. Cook over medium-low heat, stirring occasionally, until the onions are caramelized and golden brown, about 30-40 minutes.'}]

scores = get_scores(recipe_list)
scores

[0.5270266515329382, 0.5028459487913037, 0.515308160025934]

In [61]:
# nonsense recipe

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = get_scores(recipe_list)
scores

[0.5040546079674728, 0.5040292302849937, 0.5123256028930298]

# final_recipes

<br>Issues:
- Output format
    - output is 1 dict with 3 titles, 1 ingredients, 1 directions
    - dict values are in list format

In [56]:
# Most recent version from 5/13 debugging session

def final_recipes(recipe_list, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't

    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions

    UPDATES
    5/13/2024 by everyone:
    - Input format switched to recipe_list (previously: recipe_list was switched to lists of titles, ingredients, directions by code in app.py) 
    
    Inputs (3):
    recipe_list (from recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
    scores (from code in app.py) = list of 3 integers, containing scores for each recipe
    model (from get_model in app.py) = object

    Outputs (1):
    final_recipes = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for 3 final recipes

    """

    final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
    threshold = 0.3

    for i in range(len(recipe_list)):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipe_list[i]['title'])
            final_recipes["Ingredients"].append(recipe_list[i]['ingredients'])
            final_recipes["Directions"].append(recipe_list[i]['directions'])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipe_list[i]['title'],
                "Ingredients":recipe_list[i]['ingredients'],
                "Directions":recipe_list[i]['directions']
                         }
            last_recipe = {"title":[],
                           "ingredients":[],
                           "directions":[]
                         }
            while n < 3:
                new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
                new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
                if new_score[0][1] >= threshold:
                    final_recipes["Title"].append(new_recipe[0]["title"])
                    final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
                    final_recipes["Directions"].append(new_recipe[0]["directions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe[0]["title"][0][0])
                final_recipes["Ingredients"].append(last_recipe[0]["ingredients"][0][0])
                final_recipes["Directions"].append(last_recipe[0]["directions"][0][0])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

### Testing final_recipes

In [58]:
# good recipes, actual score

recipe_list = [{'title': 'Garlic Chicken Tomato Skillet',
  'ingredients': '* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  'directions': '1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.'},
 {'title': 'Simple Grilled Chicken',
  'ingredients': '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  'directions': '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 minutes per side, or until cooked through.\n4. Meanwhile, grill onion slices until caramelized and slightly charred.\n5. Serve chicken with grilled onion.'},
 {'title': 'Caramelized Onion Relish',
  'ingredients': '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced',
  'directions': '1. In a large saucepan, combine the sliced onion, salt, and garlic. Cook over medium-low heat, stirring occasionally, until the onions are caramelized and golden brown, about 30-40 minutes.'}]

scores = [0.5270266515329382, 0.5028459487913037, 0.515308160025934]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

final_recipes(recipe_list, scores, model)

{'Title': ['Garlic Chicken Tomato Skillet',
  'Simple Grilled Chicken',
  'Caramelized Onion Relish'],
 'Ingredients': ['* 1 lb chicken breast or thighs, cut into 1-inch pieces\n* 2 large tomatoes, diced\n* 3 cloves garlic, minced',
  '- 1 chicken breast\n- 1/2 onion, sliced\n- Salt, to taste',
  '* 1 large onion, thinly sliced\n* 1/2 teaspoon salt\n* 2 cloves garlic, minced'],
 'Directions': ['1. Heat a large skillet over medium-high heat.\n2. Add chicken to skillet and cook until browned, about 5-6 minutes. Remove from skillet and set aside.\n3. Add garlic to skillet and cook until fragrant, about 1 minute.\n4. Add diced tomatoes to skillet and cook until they start to soften, about 2-3 minutes.\n5. Add chicken back to skillet and stir to combine with tomato and garlic mixture. Cook until chicken is cooked through and tomato mixture is tender.',
  '1. Preheat grill to medium-high heat.\n2. Season chicken with salt, then brush both sides with a little water.\n3. Grill chicken for 6-7 

In [62]:
# nonsense recipe, actual score

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.5040546079674728, 0.5040292302849937, 0.5123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

final_recipes(recipe_list, scores, model)

{'Title': ["Gurum's Mysterious Delight", 'Recchio Croco', 'Title:'],
 'Ingredients': ['* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.',
  '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco',
  '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or un

In [64]:
# nonsense recipes with high scores

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.95040546079674728, 0.95040292302849937, 0.95123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

final_recipes(recipe_list, scores, model)

{'Title': ["Gurum's Mysterious Delight", 'Recchio Croco', 'Title:'],
 'Ingredients': ['* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.',
  '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco',
  '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or un

In [65]:
# nonsense recipes with low scores

recipe_list = [{'title': "Gurum's Mysterious Delight",
  'ingredients': '* Rugby grass (1 cup, chopped fresh)\n* Gum (1 tablespoon, dissolved)\n* Rum (2 tablespoons)\n* Mug (1, made from ceramic)',
  'directions': '1. Combine Rugby grass, gum, and rum in the mug.\n2. Stir clockwise for 7 times, then counterclockwise for 3 times.\n3. Serve immediately.'},
 {'title': 'Recchio Croco',
  'ingredients': '* RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  'directions': '1. Crumble RO Crumbs into a crock\n2. Coat Croco with Curing Oil\n3. Ooze Cream over Croco'},
 {'title': 'Title:',
  'ingredients': '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika',
  'directions': '1. Preheat oven to 375°F.\n2. In a bowl, mix together hummus, paprika, and olive oil.\n3. Dip each cherry tomato half into the hummus mixture, coating evenly.\n4. Place coated cherry tomato halves on a baking sheet.\n5. Sprinkle chopped cucumbers on top of tomato halves.\n6. Bake for 20-25 minutes, or until crispy and golden brown.\n7. Serve and enjoy!'}]

scores = [0.15040546079674728, 0.15040292302849937, 0.15123256028930298]

def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model
model = get_model()

final_recipes(recipe_list, scores, model)

{'Title': ['Mystery Drink',
  'Crunchy Creamy Fried Chicken',
  'Hummus and Veggie Wrap'],
 'Ingredients': ['* 1 cup chopped fresh Rugby grass\n* 1 tablespoon dissolved Gum\n* 2 tablespoons Rum\n* 1 ceramic Mug',
  '* * RO Crumbs\n* Curing Oil\n* Ounces of Cream',
  '* 1 cup cherry tomatoes, halved\n* 1/2 cup mashed chickpeas\n* 1/4 cup chopped fresh cucumbers\n* 2 tablespoons olive oil\n* 2 tablespoons hummus\n* 1/2 teaspoon paprika'],
 'Directions': ['1. Combine Rugby grass, Gums, and Rum in the ceramic Mug.\n2. Stir well to combine.\n3. Serve immediately.',
  '1. Dredge chicken pieces in Cream, shaking off excess.\n2. Dip cream-coated chicken pieces in * RO Crumbs, pressing gently to adhere crumbs.\n3. Heat Curing Oil in a deep frying pan to 350°F.\n4. Fry chicken pieces until golden and crispy, about 5-7 minutes.\n5. Drain fried chicken on paper towels and serve hot.',
  '1. In a medium bowl, mix together cherry tomatoes, mashed chickpeas, and chopped cucumbers.\n2. In a small bowl

#### Random stuff that may or may not be useful

In [ ]:
# Example user inputs

'chicken, mustard, coconut, orange, gravy'


In [15]:
# Example ingredients combinations

[('chicken', 'mustard'),
 ('chicken', 'coconut'),
 ('chicken', 'orange'),
 ('chicken', 'gravy'),
 ('mustard', 'coconut'),
 ('mustard', 'orange'),
 ('mustard', 'gravy'),
 ('coconut', 'orange'),
 ('coconut', 'gravy'),
 ('orange', 'gravy')]

In [ ]:
# # previous final_recipes where code in app.py changed input to lists of titles, ingredients, and directions first

# def final_recipes(recipe_dict, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
#     """
#     This evaluates whether the score of a recipe passes or fails the threshold.
#     If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
#     NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
#                         old version for this function to still work.
#                         optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    
#     Inputs (3): 
#     recipe_list (from: recipe_generator) = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for the 3 recipes
#     scores (from: get_scores) = list of 3 integers, containing scores for each recipe
#     model (from: get_model in app.py) = object 
    
#     Outputs (1): 
#     final_recipes = a list of 3 dictionaries with 3 keys each: 'title', 'ingredients', 'directions', containing info for 3 final recipes
    
#     """
#     final_recipes = {"Title": [], "Ingredients": [], "Directions": []}
#     threshold = 0.5

#     for i in range(len(recipe_dict)):
#         if scores[i] >= threshold:
#             final_recipes["Title"].append(recipe_dict[i]['title'])
#             final_recipes["Ingredients"].append(recipe_dict[i]['ingredients'])
#             final_recipes["Directions"].append(recipe_dict[i]['directions'])
#         else:
#             n = 0
#             tmp_recipe = {
#                 "Title":recipe_dict[i]['title'],
#                 "Ingredients":recipe_dict[i]['ingredients'],
#                 "Directions":recipe_dict[i]['directions']
#                          }
#             last_recipe = {"title":[],
#                            "ingredients":[],
#                            "directions":[]
#                          }
#             while n < 3:
#                 new_recipe = recipe_generator([tmp_recipe["Ingredients"]]) ###<=== insert actual recipe generator
#                 new_score = model.predict_proba([new_recipe[0]['directions']]) ###<=== insert the actual scoring model function here
#                 if new_score[0][1] >= threshold:
#                     final_recipes["Title"].append(new_recipe[0]["title"])
#                     final_recipes["Ingredients"].append(new_recipe[0]["ingredients"])
#                     final_recipes["Directions"].append(new_recipe[0]["directions"])
#                     break  # Exit loop if the new recipe passes the threshold
#                 else:
#                     last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
#                     n += 1
#             else: # Add the last generated recipe if the loop completes without finding a passing recipe
#                 final_recipes["Title"].append(last_recipe["title"][0][0])
#                 final_recipes["Ingredients"].append(last_recipe["ingredients"][0][0])
#                 final_recipes["Directions"].append(last_recipe["directions"][0][0])
#                 break  # Exit the outer loop to prevent an unending loop

#     return final_recipes